In [ ]:
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
from datasets import dataset_factory
from preprocessing import preprocessing_factory
slim = tf.contrib.slim

In [ ]:
batch_size=32

dataset_dir = "cifar10_data"
dataset = dataset_factory.get_dataset("cifar10", "test", dataset_dir)

provider = slim.dataset_data_provider.DatasetDataProvider(
    dataset,
    shuffle=False,
    common_queue_capacity=batch_size*5,
    common_queue_min=batch_size*2)
[image, label] = provider.get(['image', 'label'])

preprocessing_name = "cifarnet"
image_preprocessing_fn = preprocessing_factory.get_preprocessing(
    preprocessing_name,
    is_training=False)

eval_image_size = 32
image = image_preprocessing_fn(image, eval_image_size, eval_image_size)

images, labels = tf.train.batch(
    [image, label],
    batch_size=batch_size,
    num_threads=1,
    capacity=5 * 1)

In [ ]:
coord = tf.train.Coordinator()

gpu_options = tf.GPUOptions(allow_growth=True)
session_config = tf.ConfigProto(gpu_options=gpu_options)
with tf.Session(config=session_config) as sess:
    sess.run(tf.local_variables_initializer())
    threads = tf.train.start_queue_runners(sess, coord)
    images_eval, labels_eval = sess.run([images, labels])
    coord.request_stop()
    coord.join(threads)

In [ ]:
def to_uint8(x):
    x = 128.0 * x + 127.0
    x = np.clip(x, 0.0, 255.0)
    x  = x.astype(np.uint8)
    return x

In [ ]:
index = 0
im_list = images_eval[index]
#[0]airplane(飛行機), [1]automobile（自動車）, [2]bird（鳥）, [3]cat（猫）, [4]deer（鹿）, [5]dog（犬）, [6]frog（蛙）, [7]horse（馬）, [8]ship（船）, [9]truck（トラック）の10種類
print(labels_eval[index])
im_list = to_uint8(im_list)
plt.imshow(im_list)
plt.show()

In [ ]:
interpreter = tf.contrib.lite.Interpreter(model_path="quantized_cifarnet.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

In [ ]:
def inference_tflite(image_raw):
    tflite_input  = image_raw.reshape([1, 32, 32, 3])
    
    interpreter.set_tensor(input_details[0]['index'], tflite_input)
    interpreter.invoke()
    interpreter_output = interpreter.get_tensor(output_details[0]['index'])
    
    return np.argmax(interpreter_output)

In [ ]:
def inference_frozen(image_raw):
    tf.reset_default_graph()
    with tf.gfile.GFile("frozen_cifarnet.pb", "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as gf:
        tf.import_graph_def(graph_def, name="prefix")
        graph = gf
    
    X=graph.get_tensor_by_name('prefix/input:0')
    output_node = graph.get_tensor_by_name('prefix/CifarNet/Predictions/Softmax:0')
    with tf.Session(graph=graph) as sess:
        frozen_output=output_node.eval({X: image_raw})
        prediction = np.argmax(frozen_output)
    return prediction

In [ ]:
print("[index] truth fp32 ui8") 
for index in range(0,31):
    image_raw = images_eval[index]
    image_raw = image_raw.reshape([1,32,32,3])
    image_ui8 = to_uint8(image_raw)
    ret_tflite = inference_tflite(image_ui8)
    ret_frozen = inference_frozen(image_raw)
    print("[", index, "]", labels_eval[index], ret_frozen, ret_tflite)